## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Date
0,0,Barrow,US,71.2906,-156.7887,24.82,80,1,14.97,clear sky,2021-11-01 01:27:36
1,1,Ushuaia,AR,-54.8000,-68.3000,46.06,76,0,9.22,clear sky,2021-11-01 01:23:47
2,2,Busselton,AU,-33.6500,115.3333,56.08,62,94,18.41,overcast clouds,2021-11-01 01:25:32
3,3,Puerto Ayora,EC,-0.7393,-90.3518,69.60,90,44,7.31,scattered clouds,2021-11-01 01:27:36
4,4,Mataura,NZ,-46.1927,168.8643,61.63,49,100,10.96,overcast clouds,2021-11-01 01:27:36


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Date
5,5,Katherine,AU,-14.4667,132.2667,89.92,37,0,6.91,clear sky,2021-11-01 01:27:37
8,8,Kathu,TH,7.9178,98.3332,81.70,99,38,8.14,scattered clouds,2021-11-01 01:27:37
11,11,Port Hedland,AU,-20.3167,118.5667,80.65,57,40,10.36,scattered clouds,2021-11-01 01:27:38
13,13,Maragogi,BR,-9.0122,-35.2225,76.96,82,34,8.34,scattered clouds,2021-11-01 01:27:39
17,17,Poum,NC,-20.2333,164.0167,75.69,70,0,18.39,clear sky,2021-11-01 01:23:41
19,19,Georgetown,MY,5.4112,100.3354,81.90,87,20,4.61,few clouds,2021-11-01 01:25:41
20,20,Cockburn Town,TC,21.4612,-71.1419,82.06,69,100,7.70,overcast clouds,2021-11-01 01:27:40
26,26,Meulaboh,ID,4.1363,96.1285,77.38,84,100,3.31,light rain,2021-11-01 01:27:42
27,27,Shush,IR,32.1942,48.2436,78.75,27,100,3.89,overcast clouds,2021-11-01 01:27:42
28,28,Sola,VU,-13.8833,167.5500,80.87,74,16,14.25,few clouds,2021-11-01 01:27:43


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                188
City                   188
Country                188
Lat                    188
Lng                    188
Max Temp               188
Humidity               188
Cloudiness             188
Wind Speed             188
Weather Description    188
Date                   188
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
5,Katherine,AU,89.92,clear sky,-14.4667,132.2667,
8,Kathu,TH,81.70,scattered clouds,7.9178,98.3332,
11,Port Hedland,AU,80.65,scattered clouds,-20.3167,118.5667,
13,Maragogi,BR,76.96,scattered clouds,-9.0122,-35.2225,
17,Poum,NC,75.69,clear sky,-20.2333,164.0167,
19,Georgetown,MY,81.90,few clouds,5.4112,100.3354,
20,Cockburn Town,TC,82.06,overcast clouds,21.4612,-71.1419,
26,Meulaboh,ID,77.38,light rain,4.1363,96.1285,
27,Shush,IR,78.75,overcast clouds,32.1942,48.2436,
28,Sola,VU,80.87,few clouds,-13.8833,167.5500,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
# Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().count()

City                   188
Country                188
Max Temp               188
Weather Description    188
Lat                    188
Lng                    188
Hotel Name             188
dtype: int64

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
5,Katherine,AU,89.92,clear sky,-14.4667,132.2667,Pine Tree Motel
8,Kathu,TH,81.70,scattered clouds,7.9178,98.3332,Novotel Phuket Resort
11,Port Hedland,AU,80.65,scattered clouds,-20.3167,118.5667,The Esplanade Hotel
13,Maragogi,BR,76.96,scattered clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
17,Poum,NC,75.69,clear sky,-20.2333,164.0167,Gîte kajeon


In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure


# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))